# Shailja Somani, John Vincent Deniega, Muris Saab
# ADS 508 - Team 2
# Spring 2024

## Set-Up

### Initialize SageMaker variables
Direct copy from: 

Coyne, S. (2024, February 27). Update 01_Copy_TSV_To_S3.ipynb [Jupyter Notebook]. GitHub. Retrieved March 15, 2024, from https://github.com/MADS508/labs/blob/main/04_ingest/01_Copy_TSV_To_S3.ipynb.

In [2]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


### Import PyAthena

Coyne, S. (2022, October 9). 02_Create_Athena_Database.ipynb [Jupyter Notebook]. GitHub. Retrieved March 15, 2024, from https://github.com/MADS508/labs/commits/main/04_ingest/02_Create_Athena_Database.ipynb.

In [3]:
!pip install --disable-pip-version-check -q PyAthena==2.1.0
from pyathena import connect

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


### Create & Verify Athena Database

Coyne, S. (2022, October 9). 02_Create_Athena_Database.ipynb [Jupyter Notebook]. GitHub. Retrieved March 15, 2024, from https://github.com/MADS508/labs/commits/main/04_ingest/02_Create_Athena_Database.ipynb.

In [24]:
database_name = "covid"
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)
pd.read_sql(statement, conn)

CREATE DATABASE IF NOT EXISTS covid


""


In [25]:
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

,database_name
0,covid
1,default
2,dsoaws


## Copy 5 CSVs from Public S3 Bucket into My Private Bucket

In [67]:
# Check what is in public S3 bucket for our team
s3_public_path_csv = "s3://ads508-team2-spring24/"
!aws s3 ls $s3_public_path_csv

2024-03-16 23:56:13     592336 01_MD_COVID-19_-_Cases_by_ZIP_Code.csv
2024-03-17 22:02:37   61876919 02_SDOH_2020_ZIPCODE_1_0.csv
2024-03-16 23:56:16    3660534 03_PLACES__Local_Data_for_Better_Health__ZCTA_Data_2023_release.csv
2024-03-17 22:09:43     164479 04_Maryland_Census_Data_-_ZIP_Code_Tabulation_Areas.csv
2024-03-17 22:02:39    2564885 05_ZIPCodetoZCTACrosswalk2022UDS.csv


In [57]:
# Copy each file from the Public S3 bucket to its own folder in my private S3 bucket 
# We keep them all in their own folders because Athena external table creation takes in a full S3 folder name, not a file name

# Maryland covid cases file - list path names to file in public & where to copy in private
s3_public_path_md_covid = "s3://ads508-team2-spring24/01_MD_COVID-19_-_Cases_by_ZIP_Code.csv"
s3_priv_path_md_covid = "s3://{}/ads508-team2-spring24/md_covid_cases/01_MD_COVID-19_-_Cases_by_ZIP_Code.csv".format(bucket)

# Actually copy over
!aws s3 cp $s3_public_path_md_covid $s3_priv_path_md_covid

copy: s3://ads508-team2-spring24/01_MD_COVID-19_-_Cases_by_ZIP_Code.csv to s3://sagemaker-us-east-1-590183834230/ads508-team2-spring24/md_covid_cases/01_MD_COVID-19_-_Cases_by_ZIP_Code.csv


In [58]:
# Check that copied properly 
!aws s3 ls $s3_priv_path_md_covid

2024-03-17 21:50:06     592336 01_MD_COVID-19_-_Cases_by_ZIP_Code.csv


In [64]:
# SDOH file - list path names to file in public & where to copy in private
s3_public_path_sdoh = "s3://ads508-team2-spring24/02_SDOH_2020_ZIPCODE_1_0.csv"
s3_priv_path_sdoh = "s3://{}/ads508-team2-spring24/sdoh_2020_data/02_SDOH_2020_ZIPCODE_1_0.csv".format(bucket)

# Actually copy over
!aws s3 cp $s3_public_path_sdoh $s3_priv_path_sdoh

# Check that copied properly 
!aws s3 ls $s3_priv_path_sdoh

copy: s3://ads508-team2-spring24/02_SDOH_2020_ZIPCODE_1_0.csv to s3://sagemaker-us-east-1-590183834230/ads508-team2-spring24/sdoh_2020_data/02_SDOH_2020_ZIPCODE_1_0.csv
2024-03-17 22:05:35   61876919 02_SDOH_2020_ZIPCODE_1_0.csv


In [65]:
# CDC PLACES data file - list path names to file in public & where to copy in private
s3_public_path_cdc = "s3://ads508-team2-spring24/03_PLACES__Local_Data_for_Better_Health__ZCTA_Data_2023_release.csv"
s3_priv_path_cdc = "s3://{}/ads508-team2-spring24/cdc_places_data/03_PLACES__Local_Data_for_Better_Health__ZCTA_Data_2023_release.csv".format(bucket)

# Actually copy over
!aws s3 cp $s3_public_path_cdc $s3_priv_path_cdc

# Check that copied properly 
!aws s3 ls $s3_priv_path_cdc

copy: s3://ads508-team2-spring24/03_PLACES__Local_Data_for_Better_Health__ZCTA_Data_2023_release.csv to s3://sagemaker-us-east-1-590183834230/ads508-team2-spring24/cdc_places_data/03_PLACES__Local_Data_for_Better_Health__ZCTA_Data_2023_release.csv
2024-03-17 22:06:21    3660534 03_PLACES__Local_Data_for_Better_Health__ZCTA_Data_2023_release.csv


In [68]:
# MD Census data file - list path names to file in public & where to copy in private
s3_public_path_md_census = "s3://ads508-team2-spring24/04_Maryland_Census_Data_-_ZIP_Code_Tabulation_Areas.csv"
s3_priv_path_md_census = "s3://{}/ads508-team2-spring24/md_census_data/04_Maryland_Census_Data_-_ZIP_Code_Tabulation_Areas.csv".format(bucket)

# Actually copy over
!aws s3 cp $s3_public_path_md_census $s3_priv_path_md_census

# Check that copied properly 
!aws s3 ls $s3_priv_path_md_census

copy: s3://ads508-team2-spring24/04_Maryland_Census_Data_-_ZIP_Code_Tabulation_Areas.csv to s3://sagemaker-us-east-1-590183834230/ads508-team2-spring24/md_census_data/04_Maryland_Census_Data_-_ZIP_Code_Tabulation_Areas.csv
2024-03-17 22:10:09     164479 04_Maryland_Census_Data_-_ZIP_Code_Tabulation_Areas.csv


In [69]:
# Zip Code to ZTCA Crosswalk data file - list path names to file in public & where to copy in private
s3_public_path_xwalk = "s3://ads508-team2-spring24/05_ZIPCodetoZCTACrosswalk2022UDS.csv"
s3_priv_path_xwalk = "s3://{}/ads508-team2-spring24/crosswalk_data/05_ZIPCodetoZCTACrosswalk2022UDS.csv".format(bucket)

# Actually copy over
!aws s3 cp $s3_public_path_xwalk $s3_priv_path_xwalk

# Check that copied properly 
!aws s3 ls $s3_priv_path_xwalk

copy: s3://ads508-team2-spring24/05_ZIPCodetoZCTACrosswalk2022UDS.csv to s3://sagemaker-us-east-1-590183834230/ads508-team2-spring24/crosswalk_data/05_ZIPCodetoZCTACrosswalk2022UDS.csv
2024-03-17 22:11:13    2564885 05_ZIPCodetoZCTACrosswalk2022UDS.csv


In [83]:
# Check that everything is now in my private bucket
s3_priv_bucket = "s3://{}/ads508-team2-spring24/".format(bucket)
!aws s3 ls $s3_priv_bucket

                           PRE cdc_places_data/
                           PRE crosswalk_data/
                           PRE md_census_data/
                           PRE md_covid_cases/
                           PRE sdoh_2020_data/
2024-03-17 21:28:56     592336 01_MD_COVID-19_-_Cases_by_ZIP_Code.csv
2024-03-17 21:28:56   61503879 02_SDOH_2020_ZIPCODE_1_0.xlsx
2024-03-17 21:28:56    3660534 03_PLACES__Local_Data_for_Better_Health__ZCTA_Data_2023_release.csv
2024-03-17 22:09:01     164479 04_Maryland_Census_Data_-_ZIP_Code_Tabulation_Areas.csv
2024-03-17 21:28:56    1512369 05_ZIPCodetoZCTACrosswalk2022UDS.xlsx


## Create Athena Tables/Parquet Tables from S3 CSV Files

Coyne, S. (2022, October 9). 03_Register_S3_TSV_With_Athena.ipynb [Jupyter Notebook]. GitHub. Retrieved March 15, 2024, from https://github.com/MADS508/labs/blob/main/04_ingest/03_Register_S3_TSV_With_Athena.ipynb. \
Coyne, S. (2022, October 9). 04_Convert_S3_TSV_To_Parquet_With_Athena.ipynb [Jupyter Notebook]. GitHub. Retrieved March 17, 2024, from https://github.com/MADS508/labs/blob/main/04_ingest/04_Convert_S3_TSV_To_Parquet_With_Athena.ipynb.

In [26]:
# Set S3 path to parquet data
s3_path_parquet = "s3://{}/ads508-team2-spring24/parquet/".format(bucket)

In [27]:
# Create string variables to hold names for respective tables and the database for SQL statement
MDCovid_table = "MDCovid"
SDOH_table = "SDOH"
CDC_table = "CDC"
MDCensus_table = "MDCensus"
MDCrosswalk_table = "MDCrosswalk"
database_name = "covid"

### Tables that don't have meaningful partitions, thus don't need to be parquets for our purposes

#### MD Covid by Zip Code Table 

In [38]:
# Build SQL statement to create MD Covid by Zip Code Athena Table
table_name = MDCovid_table
path = "s3://{}/ads508-team2-spring24/md_covid_cases/".format(bucket)

'''
NOTE: The covid counts in this table are running totals, not just new cases.
However, as stated in our Assignment 2.1, to avoid the additional confounding variable of time, we will just train 
     & test our model on one week of data at a time. Thus, it's not important how the numbers for each time point are 
     calculated, as long as it remains consistent for that time point. 
There is data for every day in 2022, but we don't want to explode our this table that much, so we'll just push the 
    first 10 days of the year.
'''
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         OBJECTID int,
         ZIP_CODE int,
         total01_01_2022 int,
         total01_02_2022 int,
         total01_03_2022 int,
         total01_04_2022 int,
         total01_05_2022 int,
         total01_06_2022 int,
         total01_07_2022 int,
         total01_08_2022 int,
         total01_09_2022 int,
         total01_10_2022 int
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_name, path
)

print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS covid.MDCovid(
         OBJECTID int,
         ZIP_CODE int,
         total01_01_2022 int,
         total01_02_2022 int,
         total01_03_2022 int,
         total01_04_2022 int,
         total01_05_2022 int,
         total01_06_2022 int,
         total01_07_2022 int,
         total01_08_2022 int,
         total01_09_2022 int,
         total01_10_2022 int
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' LOCATION 's3://sagemaker-us-east-1-590183834230/ads508-team2-spring24/md_covid_cases/'
TBLPROPERTIES ('skip.header.line.count'='1')


In [39]:
# Actually create table (execute statement)
pd.read_sql(statement, conn)

""


In [33]:
# Check that worked
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

,tab_name
0,mdcovid


In [37]:
# Test that data is stored properly
pd.read_sql("SELECT * FROM covid.MDCovid LIMIT 10", conn).head(10)

,objectid,zip_code,total01_01_2022,total01_02_2022,total01_03_2022,total01_04_2022,total01_05_2022,total01_06_2022,total01_07_2022,total01_08_2022,total01_09_2022,total01_10_2022
0,517,20010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,518,20015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,519,20037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,520,20074,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,521,20161,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,522,20185,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,523,20437,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,525,20602,3755.0,3834.0,3941.0,4014.0,4055.0,4105.0,4140.0,4214.0,4292.0,4323.0
8,526,20603,3851.0,3946.0,4049.0,4127.0,4185.0,4277.0,4328.0,4441.0,4555.0,4609.0
9,527,20604,55.0,56.0,57.0,59.0,60.0,61.0,61.0,61.0,63.0,64.0


#### Maryland Census Data Table

In [56]:
table_name = MDCensus_table
path = "s3://{}/ads508-team2-spring24/md_census_data/".format(bucket)

statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         OBJECTID_1 int,
         ZCTA5CE10 int,
         FIRST_STAT int,
         FIRST_GEOI int,
         FIRST_CLAS CHAR(2),
         FIRST_MTFC string,
         FIRST_FUNC CHAR(1),
         ZCTA5N int,
         STATE int,
         AREALAND int,
         AREAWATR int,
         POP100 int,
         HU100 int,
         NHW int,
         NHB int,
         NHAI int,
         NHA int,
         NHNH int,
         NHO int,
         NHT int,
         HISP int,
         PNHW DECIMAL(3,1),
         PNHB DECIMAL(3,1),
         PNHAI DECIMAL(2,1),
         PNHA DECIMAL(2,1),
         PNHNH DECIMAL(2,1),
         PNHO DECIMAL(2,1),
         PNHT DECIMAL(2,1),
         PHISP DECIMAL(2,1),
         POP65_ int,
         PCTPOP65_ DECIMAL(3,1),
         MEDAGE DECIMAL(3,1)
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_name, path
)

print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS covid.MDCensus(
         OBJECTID_1 int,
         ZCTA5CE10 int,
         FIRST_STAT int,
         FIRST_GEOI int,
         FIRST_CLAS CHAR(2),
         FIRST_MTFC string,
         FIRST_FUNC CHAR(1),
         ZCTA5N int,
         STATE int,
         AREALAND int,
         AREAWATR int,
         POP100 int,
         HU100 int,
         NHW int,
         NHB int,
         NHAI int,
         NHA int,
         NHNH int,
         NHO int,
         NHT int,
         HISP int,
         PNHW DECIMAL(3,1),
         PNHB DECIMAL(3,1),
         PNHAI DECIMAL(2,1),
         PNHA DECIMAL(2,1),
         PNHNH DECIMAL(2,1),
         PNHO DECIMAL(2,1),
         PNHT DECIMAL(2,1),
         PHISP DECIMAL(2,1),
         POP65_ int,
         PCTPOP65_ DECIMAL(3,1),
         MEDAGE DECIMAL(3,1)
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' LOCATION 's3://sagemaker-us-east-1-590183834230/ads508-team2-spring24/md_census_data/'
TBLPROPERTIES ('sk

In [57]:
# Actually run query
pd.read_sql(statement, conn)

""


In [58]:
# Take a look at table to test 
df_show = pd.read_sql("SELECT * FROM covid.MDCensus LIMIT 5", conn)
df_show.head()

,objectid_1,zcta5ce10,first_stat,first_geoi,first_clas,first_mtfc,first_func,zcta5n,state,arealand,...,pnhb,pnhai,pnha,pnhnh,pnho,pnht,phisp,pop65_,pctpop65_,medage
0,1,20601,24,2420601,B5,G6350,S,20601,24,115635266,...,46.1,0.6,3.6,0.0,0.2,3.5,5.3,1922,8.0,37.3
1,2,20602,24,2420602,B5,G6350,S,20602,24,35830723,...,52.3,0.5,2.9,0.1,0.2,4.0,6.1,1964,7.9,32.6
2,3,20603,24,2420603,B5,G6350,S,20603,24,44239637,...,51.9,0.3,5.0,0.1,0.2,3.8,5.5,1400,4.8,34.5
3,4,20606,24,2420606,B5,G6350,S,20606,24,7501011,...,10.4,0.2,0.9,0.0,0.0,0.0,0.9,108,25.1,49.1
4,5,20607,24,2420607,B5,G6350,S,20607,24,54357590,...,64.5,0.3,5.7,0.0,0.1,2.4,4.9,847,8.6,40.9


#### ZTCA to Zip Code Crosswalk Table

In [62]:
table_name = MDCrosswalk_table
path = "s3://{}/ads508-team2-spring24/crosswalk_data/".format(bucket)

#TO DO: Figure out how to handle total{date} columns for schema. Columnar?
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         ZIP_CODE int,
         PO_NAME string,
         STATE CHAR(2),
         ZIP_TYPE string,
         zcta int,
         zip_join_type string
)ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_name, path
)

print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS covid.MDCrosswalk(
         ZIP_CODE int,
         PO_NAME string,
         STATE CHAR(2),
         ZIP_TYPE string,
         zcta int,
         zip_join_type string
)ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' LOCATION 's3://sagemaker-us-east-1-590183834230/ads508-team2-spring24/crosswalk_data/'
TBLPROPERTIES ('skip.header.line.count'='1')


In [63]:
# Actually run query
pd.read_sql(statement, conn)

# Take a look at table to test 
df_show = pd.read_sql("SELECT * FROM covid.MDCrosswalk LIMIT 5", conn)
df_show.head()

,zip_code,po_name,state,zip_type,zcta,zip_join_type
0,501,Holtsville,NY,Post Office or large volume customer,11742,Spatial join to ZCTA
1,544,Holtsville,NY,Post Office or large volume customer,11742,Spatial join to ZCTA
2,601,Adjuntas,PR,Zip Code Area,601,Zip matches ZCTA
3,602,Aguada,PR,Zip Code Area,602,Zip matches ZCTA
4,603,Aguadilla,PR,Zip Code Area,603,Zip matches ZCTA


### Tables that have Meaningful Partitions, Thus should be Parquets

### Social Determinants of Health Database Table

In [10]:
table_name = SDOH_table
path = SDOH_path

#TO DO: Figure out how to handle the 327 columns. S3?
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         OBJECTID int,
         ZIP_CODE int,
)""".format(
    database_name, table_name, path
)

print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS covid.SDOH(
         OBJECTID int,
         ZIP_CODE int,
)


### PLACES: Local Data for Better Health Table

In [11]:
table_name = CDC_table
path = CDC_path

#TO DO: Figure out how to handle total{date} columns for schema. Columnar?
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         YEAR YEAR,
         LocationName int,
         DataSource string,
         Category string,
         Measure string,
         Data_Value_Unit string,
         Data_Value_Type string,
         Data_Value DECIMAL(3,1),
         Data_Value_Footnote_Symbol string,
         Data_Value_Footnote string,
         Low_Confidence_Limit DECIMAL(3,1),
         High_Confidence_Limit DECIMAL(3,1),
         TotalPopulation int,
         Geolocation string,
         LocationID int,
         CategoryID string,
         MeasureId string,
         DataValueTypeID string,
         Short_Question_Text string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_name, path
)

print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS covid.CDC(
         YEAR YEAR,
         LocationName int,
         DataSource string,
         Category string,
         Measure string,
         Data_Value_Unit string,
         Data_Value_Type string,
         Data_Value DECIMAL(3,1),
         Data_Value_Footnote_Symbol string,
         Data_Value_Footnote string,
         Low_Confidence_Limit DECIMAL(3,1),
         High_Confidence_Limit DECIMAL(3,1),
         TotalPopulation int,
         Geolocation string,
         LocationID int,
         CategoryID string,
         MeasureId string,
         DataValueTypeID string,
         Short_Question_Text string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' LOCATION 's3://ads508-team2-spring24/03_PLACES__Local_Data_for_Better_Health__ZCTA_Data_2023_release.csv'
TBLPROPERTIES ('skip.header.line.count'='1')


### Check all tables are in Covid database

In [6]:
statement = "SHOW TABLES FROM covid"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

,tab_name
0,mdcensus
1,mdcovid


## Release Resources: Direct copy from

Coyne, S. (2024, February 27). Update 01_Copy_TSV_To_S3.ipynb [Jupyter Notebook]. GitHub. https://github.com/MADS508/labs/blob/main/04_ingest/01_Copy_TSV_To_S3.ipynb

In [14]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [15]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>